In [ ]:
import csv
import pickle
import re
import time
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import swifter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pymongo import MongoClient
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer

# Configuration
### Select parameters for cleaning

In [ ]:
ngram_length = 3
min_yearly_df = 3
start_year = 7
end_year = 22

# Cleaning pipeline

In [ ]:
alphabets = "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
htmltags = '<[^>]+>'
htmlspecial = '&#?[xX]?[a-zA-Z0-9]{2,8};'

start_delimiter = 'documentstart'
sent_delimiter = 'sentenceboundary'
end_delimiter = 'documentend'

delimiters = [start_delimiter, sent_delimiter, end_delimiter]

# Download the lemmatisesr
wnl = WordNetLemmatizer()

# Create a tokeniser
count = CountVectorizer(strip_accents='ascii', min_df=1)
tokeniser = count.build_analyzer()


def normalise_acronymns(text):
    '''
    Remove the periods in acronyms. 
    Adapted from the method found at https://stackoverflow.com/a/40197005 
    '''
    return re.sub(r'(?<!\w)([A-Z, a-z])\.', r'\1', text)


def normalise_decimals(text):
    '''
    Remove the periods in decimal numbers and replace with POINT
    '''
    return re.sub(r'([0-9])\.([0-9])', r'\1POINT\2', text)


def split_into_sentences(text):
    '''
    Sentence splitter adapted from https://stackoverflow.com/a/31505798
    '''
    text = text.replace("\n", " ")
    text = re.sub(prefixes, "\\1<prd>", text)
    text = re.sub(websites, "<prd>\\1", text)

    # my addition
    text = re.sub(htmltags, " ", text)
    text = re.sub(htmlspecial, " ", text)

    if "Ph.D" in text:
        text = text.replace("Ph.D.", "PhD")

    text = re.sub("\s" + alphabets + "[.] ", " \\1", text)
    text = re.sub(acronyms+" "+starters, "\\1<stop> \\2", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]", "\\1\\2\\3", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]", "\\1\\2", text)
    text = re.sub(" "+suffixes+"[.] "+starters, " \\1 \\2", text)
    text = re.sub(" "+suffixes+"[.]", " \\1", text)
    text = re.sub(" " + alphabets + "[.]", " \\1", text)

    if "”" in text:
        text = text.replace(".”", "”.")
    if "\"" in text:
        text = text.replace(".\"", "\".")
    if "!" in text:
        text = text.replace("!\"", "\"!")
    if "?" in text:
        text = text.replace("?\"", "\"?")

    text = text.replace(".", "<stop>")
    text = text.replace("?", "<stop>")
    text = text.replace("!", "<stop>")

    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]

    non_empty = []

    for s in sentences:
        # we require that there be two alphanumeric characters in a row
        if len(re.findall("[A-Za-z0-9][A-Za-z0-9]", s)) > 0:
            non_empty.append(s)
    return non_empty


def pad_sentences(sentences):
    '''
    Takes a list of sentences and returns a string in which:
        - The beginning of the abstract is indicated by DOCUMENTSTART
        - The end is indicated by DOCUMENTEND
        - Sentence boundaries are indicated by SENTENCEBOUNDARY

    The number of delimiters used is dependent on the ngram length
    '''
    sent_string = (' '+(sent_delimiter+' ')*(ngram_length-1)).join(sentences)

    return (start_delimiter+' ')*(ngram_length-1) + sent_string + (' '+end_delimiter)*(ngram_length-1)


def get_stopwords():
    stop = set(stopwords.words('english'))
    stop = set([s.replace("'", "") for s in stop])

    # Add years to prevent spikes
    for year in range(1900, 2020):
        stop.add(str(year))

    # Add small numbers
    for num in range(0, 100):
        if len(str(num)) < 2:
            stop.add(str(num))
            num = '0' + str(num)

        stop.add(str(num))

    # Add these extra stopwords to the list
    # TODO: Look through the corpus and decide which are
    # extra stopwords needed for this specific domain
    extra = [
        'use', 'using', 'uses', 'used', 'based', 'including', 'include', 'approach', 'factors', 'business', 'risk'
        'wa', 'ha', 'doe', 'item', '1a', 'factor', '1b', '1aitem', '10-k', '1AITEM', 'could', 'regarding'
    ]
    for word in extra:
        stop.add(word)
    return stop


def cleaning_pipeline(text):
    '''
    Takes a binary string and returns a list of cleaned sentences, stripped of punctuation and lemmatised
    '''

    stopwords = get_stopwords()
    text = normalise_decimals(normalise_acronymns(text))
    text = ' '.join([word for word in text.split() if word not in stopwords])
    sentences = split_into_sentences(text)

    # strip out punctuation and make lowercase
    clean_sentences = []
    for s in sentences:

        # Deal with special cases
        s = re.sub(r'[-/]', ' ', s)

        # Remove all other punctuation
        s = re.sub(r'[^\w\s]', '', s)

        clean_sentences.append(s.lower())

    # pad sentences with delimiters
    text = pad_sentences(clean_sentences)

    # Lemmatise word by word
    lemmas = []
    for word in tokeniser(text):
        lemmas.append(wnl.lemmatize(word))

    return ' '.join(lemmas)


def cleaning_pipeline_df(row):
    row['clean_text'] = cleaning_pipeline(row['text'])
    return row


# Centralise data into a csv

In [ ]:
# Connect to MongoDB
client = MongoClient('127.0.0.1', 27017)
db = client.frtp
collection = db.documents

In [ ]:
test_find = collection.find({'ticker':'MSFT'})

In [ ]:
test_find = pd.DataFrame(list(test_find))

In [ ]:
test_find.head()

In [ ]:
test_find['clean_text'] = ''
test_find['stop_clean_text'] = ''
test_find = test_find.swifter.apply(cleaning_pipeline_df,axis=1)

In [ ]:
all_years_df = pd.DataFrame()
for year in tqdm(range(start_year, end_year)):
    result = collection.find({"year": f"{year:02d}"})
    df = pd.DataFrame(list(result))
    df['clean_text'] = ''
    df = df.swifter.apply(cleaning_pipeline_df, axis=1)
    df.to_csv(f'data/clean_data/{year}.csv')
    all_years_df = all_years_df.append(df)
all_years_df.to_csv('all_years_df.csv')


# Build a vocabulary

In [ ]:
vocab = set()
for year in range(start_year, end_year):
    df = pd.read_csv(f'data/clean_data/{year}.csv')
    # The same as above, applied year by year instead.
    t0 = time.time()

    vectorizer = CountVectorizer(strip_accents='ascii',
                                 ngram_range=(1, ngram_length),
                                 stop_words=get_stopwords(),
                                 min_df=min_yearly_df)

    vector = vectorizer.fit_transform(df.clean_text)

    # Save the new words
    vocab = vocab.union(vectorizer.vocabulary_.keys())

    print(year, len(vocab), time.time()-t0)


vocabulary = {}
i = 0
for v in vocab:
    # Remove delimiters
    if start_delimiter in v:
        pass
    elif end_delimiter in v:
        pass
    elif sent_delimiter in v:
        pass
    else:
        vocabulary[v] = i
        i += 1

print(len(vocabulary.keys()))


In [ ]:
vectors = []
for year in range(start_year, end_year):
    df = pd.read_csv(f'data/clean_data/{year}.csv')

    # The same as above, applied year by year instead.
    t0 = time.time()

    vectorizer = CountVectorizer(strip_accents='ascii',
                                 ngram_range=(1, ngram_length),
                                 stop_words=get_stopwords(),
                                 vocabulary=vocabulary)

    vectors.append(vectorizer.fit_transform(df.clean_text))

    print(year, time.time()-t0)


In [ ]:
summed_vectors = []

for y in range(len(vectors)):
    vector = vectors[y]

    # Set all elements that are greater than one to one -- we do not care if a word is used multiple times in
    # the same document
    vector[vector > 1] = 1

    # Sum the vector along columns
    summed = np.squeeze(np.asarray(np.sum(vector, axis=0)))

    # Normalise by dividing by the number of documents in that year
    normalised = summed/vector.shape[0]

    # Save the summed vector
    summed_vectors.append(normalised)

# Stack vectors vertically, so that we have the full history of popularity/time for each term
stacked_vectors = np.stack(summed_vectors, axis=1)

print(stacked_vectors.shape)

stacked_vectors = pd.DataFrame(stacked_vectors.transpose(), columns=list(vocabulary.keys()))


In [ ]:
normalisation = stacked_vectors.sum(axis=1)
stacked_vectors = stacked_vectors.divide(normalisation, axis=0)*100

In [ ]:
long_ma_length = 8
short_ma_length = 4
signal_line_ma = 2
significance_ma_length = 2

significance_threshold = 0.0005
years_above_significance = 2
testing_period = 2

# Detection threshold is set such that the top 500 terms are chosen
burstiness_threshold_prediction = 0.0005
burstiness_threshold_detection = 0.00049

In [ ]:
def calc_macd(dataset):
    long_ma = dataset.ewm(span=long_ma_length).mean()
    short_ma = dataset.ewm(span=short_ma_length).mean()
    significance_ma = dataset.ewm(span=significance_ma_length).mean()
    macd = short_ma - long_ma
    signal = macd.ewm(span=signal_line_ma).mean()
    hist = macd - signal
    return long_ma, short_ma, significance_ma, macd, signal, hist

def calc_significance(stacked_vectors, significance_threshold, n):
    # Must have been above the significance threshold for two consecutive timesteps
    a = stacked_vectors>significance_threshold
    b = a.rolling(window=n).sum()
    return stacked_vectors[stacked_vectors.axes[1][np.where(b.max()>=n)[0]]]
    
def calc_burstiness(hist, scaling_factor):
    return hist.iloc[long_ma_length-1:]/scaling_factor

def calc_scaling(significance_ma, method):
    if method == "max":
        scaling = significance_ma.iloc[significance_ma_length-1:].max()
    elif method == "mean":
        scaling = significance_ma.iloc[significance_ma_length-1:].mean()
    elif method == "sqrt":
        scaling = np.sqrt(significance_ma.iloc[significance_ma_length-1:].max())
    else:
        raise ValueError("No valid method provided")      
    return scaling

def max_burstiness(burstiness, absolute=False):
    if absolute:
        b = pd.concat([np.abs(burstiness).max(), burstiness.idxmax()], axis=1)
    else:
        try:
            b = pd.concat([burstiness.max(), burstiness.idxmax()], axis=1)
        except ValueError:
            print(burstiness)
            return None
    b.columns = ["max", "location"]
    return b


In [ ]:
stacked_vectors = calc_significance(stacked_vectors, significance_threshold, years_above_significance)
print(stacked_vectors.shape)

In [ ]:
long_ma, short_ma, significance_ma, macd, signal, hist = calc_macd(stacked_vectors)
scaling_factor = calc_scaling(significance_ma, "sqrt")
burstiness_over_time = calc_burstiness(hist, scaling_factor)
burstiness = max_burstiness(burstiness_over_time)

In [ ]:
print(np.sum(burstiness["max"]>0.0005))

In [ ]:
bursts = list(burstiness["max"].index[np.where(burstiness["max"]>burstiness_threshold_detection)[0]])
print(bursts)

In [ ]:
# vectorise again, using these terms only
vectors = []
for year in range(start_year, end_year):
    df = pd.read_csv(f'data/clean_data/{year}.csv')
    
    # The same as above, applied year by year instead.
    t0 = time.time()


    vectorizer = CountVectorizer(strip_accents='ascii', 
                                 ngram_range=(1,ngram_length),
                                 stop_words=get_stopwords(),
                                 vocabulary=bursts)

    vector = vectorizer.fit_transform(df.clean_text)
    
    # If any element is larger than one, set it to one
    vector.data = np.where(vector.data>0, 1, 0)
    
    vectors.append(vector)
    
    print(year, time.time()-t0)


In [ ]:
cooccurrence = []
for v in vectors:
    c = v.T*v
    c.setdiag(0)
    c = c.todense()
    cooccurrence.append(c)
    
all_cooccurrence = np.sum(cooccurrence, axis=0)

# Translate co-occurence into a distance
dists = 1- all_cooccurrence/all_cooccurrence.max()

# Remove the diagonal (squareform requires diagonals be zero)
dists -= np.diag(np.diagonal(dists))

# Put the distance matrix into the format required by hierachy.linkage
flat_dists = squareform(dists)

# Get the linkage matrix
linkage_matrix = hierarchy.linkage(flat_dists, "ward")

assignments = hierarchy.fcluster(linkage_matrix, 100, 'maxclust')
print(len(bursts))
print(len(set(assignments)))

clusters = defaultdict(list)

for term, assign in zip(bursts, assignments):
    clusters[assign].append(term)
    
for key in sorted(clusters.keys()):
    print(key, ':',  ', '.join(clusters[key]))


In [ ]:
cluster_df = pd.DataFrame({
    'cluster_id':pd.Series(dtype=int),
    'cluster_terms':pd.Series(dtype=str)
})
for k,v in clusters.items():
    terms = ';'.join(v)
    entry ={
        'cluster_id':k,
        'cluster_terms':terms
    }
    cluster_df = cluster_df.append(entry,ignore_index=True)
cluster_df.to_csv('clusters/clusters.csv')
cluster_df

# Manually clean cluster - in a separate CSV

In [ ]:
df = pd.read_csv('clusters/clusters.csv')
clusters = [d.split(';') for d in df.cluster_terms]

In [ ]:
clusters

In [ ]:
def feature_selection(dataset):
    '''
    Compile the features for the prediction step
    '''
    long_ma = dataset.ewm(span=long_ma_length).mean()
    short_ma = dataset.ewm(span=short_ma_length).mean()
    significance_ma = dataset.ewm(span=significance_ma_length).mean()
    macd = short_ma - long_ma
    signal = macd.ewm(span=signal_line_ma).mean()
    hist = macd - signal
    
    scaling_factor = calc_scaling(significance_ma, "sqrt")
    burstiness_over_time = calc_burstiness(hist, scaling_factor)
    burstiness = max_burstiness(burstiness_over_time)
    
    
    X = long_ma.iloc[long_ma_length:].T
    scaled_hist = hist.iloc[long_ma_length:]/scaling_factor
    scaled_signal = signal.iloc[long_ma_length:]/scaling_factor
 
    Xtra = pd.concat([significance_ma.iloc[-1], 
                      dataset.iloc[-1],
                        significance_ma.iloc[significance_ma_length:].std()/scaling_factor,
                        significance_ma.iloc[significance_ma_length:].max(),
                        significance_ma.iloc[significance_ma_length:].min(),
                      scaling_factor
                        ], axis=1)
    X = pd.concat([X,scaled_hist.T,scaled_signal.T,Xtra], axis=1)

    X.columns = [str(i) for i in range(6)] + ["hist"+str(i) for i in range(6)] + ["signal"+str(i) for i in range(6)] + [
                        "significance",
                        "prevalence",
                        "scaled std",
                        "max",
                        "min",
                        "scaling"
                    ]


    return X


### Prediction

In [ ]:
year_range = list(range(2010,2020))
development_data = {}
for year in range(2015, 2020):
    year_idx = year_range.index(year)

    print(year)
    # Use our three-year method to calc significance
    valid_vectors = calc_significance(stacked_vectors[:year_idx+1], significance_threshold, 2)
    
    # Recalculate the macd things based on this more limited dataset
    long_ma, short_ma, significance_ma, macd, signal, hist = calc_macd(valid_vectors)
    print(long_ma.shape,short_ma.shape,significance_ma.shape,macd.shape,signal.shape,hist.shape)
    
    
    # Calculate scaling factor
    scaling_factor = calc_scaling(significance_ma.iloc[max(long_ma_length, year_idx-8):year_idx+1], "sqrt")
    print(scaling_factor.shape)
    
    # Calculate the burstiness
    burstiness_over_time = calc_burstiness(hist, scaling_factor)
    print("Burstiness over time")
    print(burstiness_over_time.shape)
    burstiness = max_burstiness(burstiness_over_time)
    # print(burstiness.shape)
    # Choose terms that are above both thresholds (burstiness, and also most recent year was significant)
    print(np.mean(burstiness["max"]))
    print(np.std(burstiness["max"]))
    print(np.std(significance_ma))
    burst_idx = np.where((burstiness["max"]>0.00005)&(significance_ma.iloc[year_idx]>significance_threshold))[0]
    
    # Find the actual names of these terms
    bursts = valid_vectors.keys()[burst_idx]
    
    # Create a new, much smaller dataset
    dataset = stacked_vectors[bursts].iloc[year_idx-5:year_idx+1]
    
    # Get the scaled y values
    if year < 2018:
        y = stacked_vectors[bursts].iloc[year_idx+testing_period]


    # Select features and store the data
    development_data[year] = {}
    development_data[year]["X"] = feature_selection(dataset)
    if year < 2018:
        development_data[year]["y"]=y-development_data[year]["X"]['significance']
    print(year, len(bursts))


In [ ]:
X = np.array(pd.concat([development_data[year]["X"] for year in range(2008,2015)]))
y = np.array(pd.concat([development_data[year]["y"] for year in range(2008,2015)]))
y_thresh = np.zeros_like(y)
y_thresh[y>0] = 1

# Balance the sample
X, y_thresh = balanced_subsample(X, y_thresh,subsample_size=1.0)
x_train, x_test, y_train, y_test = train_test_split(X, y_thresh, test_size=0.33, random_state=1)

max_depths = np.linspace(1, 32, 32, endpoint=True)
train_results = []
test_results = []
for max_depth in max_depths:
    rf = RandomForestClassifier(n_estimators=100, max_depth=max_depth)
    rf.fit(x_train, y_train)
    train_pred = rf.predict(x_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = rf.predict(x_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)


line1, = plt.plot(max_depths, train_results, 'b', label="Train AUC")
line2, = plt.plot(max_depths, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('Tree depth')
plt.title('Effect of changing the maximum depth of the random forest on classifier accuracy')
plt.show()

In [ ]:
X = np.array(pd.concat([development_data[year]["X"] for year in range(2008,2015)]))
y = np.array(pd.concat([development_data[year]["y"] for year in range(2008,2015)]))
y_thresh = np.zeros_like(y)
y_thresh[y>0] = 1

# Balance the sample
X, y_thresh = balanced_subsample(X, y_thresh,subsample_size=1.0)
x_train, x_test, y_train, y_test = train_test_split(X, y_thresh, test_size=0.33, random_state=3)

n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 150, 200, 300, 500, 800]
train_results = []
test_results = []
for estimator in n_estimators:
    rf = RandomForestClassifier(n_estimators=estimator, max_depth=13, n_jobs=-1)
    rf.fit(x_train, y_train)
    train_pred = rf.predict(x_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = rf.predict(x_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(n_estimators, train_results, 'b', label='Train AUC')
line2, = plt.plot(n_estimators, test_results, 'r', label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.title('Effect of changing the number of estimators of the random forest on classifier accuracy')
plt.show()
